In [13]:
import requests
from bs4 import BeautifulSoup

# スクレイピング関数
def scrape_accommodations(url):
    response = requests.get(url)
    
    # エンコーディングを明示的に設定
    response.encoding = 'Windows-31J'  # エンコーディングを指定
    soup = BeautifulSoup(response.content, 'html.parser')  # バイナリデータを使用

    # 必要なデータを格納するリスト
    accommodations = []

    # 宿泊施設情報を取得
    for item in soup.select('.p-searchResultItem'):  # 各宿泊施設の要素を取得
        try:
            name = item.select_one('.p-searchResultItem__facilityName').get_text(strip=True)
            stars = float(item.select_one('.p-searchResultItem__summaryaverage-num').get_text(strip=True))  # 星数
            price = item.select_one('.p-searchResultItem__lowestPriceValue').get_text(strip=True)  # 値段
            accommodations.append({'name': name, 'stars': stars, 'price': price})
        except AttributeError:
            continue  # 情報が欠けている場合はスキップ
    return accommodations

# ホテルのデータを取得するURL
hotel_url = "https://www.jalan.net/090000/LRG_090200/?dateUndecided=1&stayCount=1&roomCount=1&adultNum=2&minPrice=0&maxPrice=999999&yadHb=1&distCd=01&kenCd=090000&lrgCd=090200&roomCrack=200000&careOnsen=1"

# データの取得
hotel_data = scrape_accommodations(hotel_url)

# URLリスト
urls = [
    "https://www.jalan.net/090000/LRG_090200/?screenId=UWW1402&distCd=01&activeSort=0&mvTabFlg=1&stayYear=&stayMonth=&stayDay=&stayCount=1&roomCount=1&dateUndecided=1&adultNum=2&yadRk=1&careOnsen=1&roomCrack=200000&kenCd=090000&lrgCd=090200&vosFlg=6&idx=0",
    "https://www.jalan.net/090000/LRG_090200/page2.html?screenId=UWW1402&distCd=01&activeSort=0&mvTabFlg=1&stayYear=&stayMonth=&stayDay=&stayCount=1&roomCount=1&dateUndecided=1&adultNum=2&yadRk=1&careOnsen=1&roomCrack=200000&kenCd=090000&lrgCd=090200&vosFlg=6&idx=30",
    "https://www.jalan.net/090000/LRG_090200/page3.html?screenId=UWW1402&distCd=01&activeSort=0&mvTabFlg=1&stayYear=&stayMonth=&stayDay=&stayCount=1&roomCount=1&dateUndecided=1&adultNum=2&yadRk=1&careOnsen=1&roomCrack=200000&kenCd=090000&lrgCd=090200&vosFlg=6&idx=60"
]

# データ取得
ryokan_data_all_pages = []
for url in urls:  # 各ページURLを順番に処理
    print(f"Scraping: {url}")  # 現在処理中のURLを表示
    ryokan_data = scrape_accommodations(url)
    ryokan_data_all_pages.extend(ryokan_data)  # 各ページのデータを結合

# 重複削除
full_ryokan_data = {entry['name']: entry for entry in ryokan_data_all_pages}.values()

# 結果を表示
print("ホテルデータ:", hotel_data)
print("旅館データ:", list(full_ryokan_data))



Scraping: https://www.jalan.net/090000/LRG_090200/?screenId=UWW1402&distCd=01&activeSort=0&mvTabFlg=1&stayYear=&stayMonth=&stayDay=&stayCount=1&roomCount=1&dateUndecided=1&adultNum=2&yadRk=1&careOnsen=1&roomCrack=200000&kenCd=090000&lrgCd=090200&vosFlg=6&idx=0
Scraping: https://www.jalan.net/090000/LRG_090200/page2.html?screenId=UWW1402&distCd=01&activeSort=0&mvTabFlg=1&stayYear=&stayMonth=&stayDay=&stayCount=1&roomCount=1&dateUndecided=1&adultNum=2&yadRk=1&careOnsen=1&roomCrack=200000&kenCd=090000&lrgCd=090200&vosFlg=6&idx=30
Scraping: https://www.jalan.net/090000/LRG_090200/page3.html?screenId=UWW1402&distCd=01&activeSort=0&mvTabFlg=1&stayYear=&stayMonth=&stayDay=&stayCount=1&roomCount=1&dateUndecided=1&adultNum=2&yadRk=1&careOnsen=1&roomCrack=200000&kenCd=090000&lrgCd=090200&vosFlg=6&idx=60
ホテルデータ: [{'name': '草津温泉\u3000ホテルヴィレッジ', 'stars': 4.4, 'price': '19,800円〜'}, {'name': 'ラビスタ草津ヒルズ（共立リゾート）', 'stars': 4.6, 'price': '43,000円〜'}, {'name': '草津ナウリゾートホテル', 'stars': 4.1, 'price': '33,00

In [14]:
import sqlite3

# ホテル用データベースを作成
def save_hotel_data(data):
    conn = sqlite3.connect("hotel_data.db")
    cursor = conn.cursor()

    # テーブル作成
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS hotels (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT,
        price TEXT,
        stars REAL
    )
    """)

    # 辞書をタプルに変換
    data_for_insertion = [(hotel["name"], hotel["price"], hotel["stars"]) for hotel in data]

    # データベースに挿入
    cursor.executemany("""
        INSERT INTO hotels (name, price, stars)
        VALUES (?, ?, ?)
    """, data_for_insertion)

    # コミットして変更を保存
    conn.commit()
    conn.close()

# 例: ホテルデータを保存
save_hotel_data(hotel_data)



In [15]:
# 旅館用データベースを作成
def save_ryokan_data(data):
    conn = sqlite3.connect("ryokan_data.db")
    cursor = conn.cursor()

    # テーブル作成
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS ryokans (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT,
        price TEXT,
        stars REAL
    )
    """)

    # 辞書をタプルに変換
    data_for_insertion = [(ryokan["name"], ryokan["price"], ryokan["stars"]) for ryokan in data]

    # データ挿入
    cursor.executemany("""
    INSERT INTO ryokans (name, price, stars)
    VALUES (?, ?, ?)
    """, data_for_insertion)

    conn.commit()
    conn.close()

# 例: 旅館データを保存
save_ryokan_data(ryokan_data)


In [16]:
# データを挿入する関数
def insert_hotel_data(hotel_data):
    conn = sqlite3.connect("hotel_data.db")
    cursor = conn.cursor()

    for hotel in hotel_data:
        # すでに存在する場合は挿入しない（重複を防ぐ）
        cursor.execute('''
        INSERT OR IGNORE INTO hotels (name, stars, price) 
        VALUES (?, ?, ?)
        ''', (hotel['name'], hotel['stars'], hotel['price']))

    # コミットして変更を保存
    conn.commit()
    conn.close()

# データベースに挿入
insert_hotel_data(hotel_data)


In [17]:
# データを挿入する関数
def insert_ryokan_data(ryokan_data):
    conn = sqlite3.connect("ryokan_data.db")
    cursor = conn.cursor()

    for ryokan in ryokan_data:
        # すでに存在する場合は挿入しない（重複を防ぐ）
        cursor.execute('''
        INSERT OR IGNORE INTO ryokans (name, stars, price) 
        VALUES (?, ?, ?)
        ''', (ryokan['name'], ryokan['stars'], ryokan['price']))

    # コミットして変更を保存
    conn.commit()
    conn.close()

# データベースに挿入
insert_ryokan_data(full_ryokan_data)

In [18]:
import pandas as pd

# ホテルデータを読み込む関数
def load_hotel_data():
    conn = sqlite3.connect("hotel_data.db")
    query = "SELECT * FROM hotels"
    df = pd.read_sql(query, conn)
    conn.close()
    return df

# 旅館データを読み込む関数
def load_ryokan_data():
    conn = sqlite3.connect("ryokan_data.db")
    query = "SELECT * FROM ryokans"
    df = pd.read_sql(query, conn)
    conn.close()
    return df

# ホテルデータの表示
def print_hotel_data():
    df = load_hotel_data()
    print("Hotel Data:")
    print(df)  # データフレームを表示

# 旅館データの表示
def print_ryokan_data():
    df = load_ryokan_data()
    print("Ryokan Data:")
    print(df)  # データフレームを表示

# ホテルと旅館データを表示
print_hotel_data()
print_ryokan_data()


Hotel Data:
    id                    name     price  stars
0    1           草津温泉　ホテルヴィレッジ  19,800円〜    4.4
1    2       ラビスタ草津ヒルズ（共立リゾート）  43,000円〜    4.6
2    3             草津ナウリゾートホテル  33,000円〜    4.1
3    4           草津温泉　ホテルヴィレッジ  19,800円〜    4.4
4    5                草津温泉　VAN  66,480円〜    4.8
5    6             亀の井ホテル 草津湯畑  25,800円〜    4.1
6    7  フォートリート草津〈2023年4月オープン〉  19,800円〜    4.3
7    8               草津温泉　ふたばや  12,600円〜    4.8
8    9          リブマックスリゾート草津温泉  23,800円〜    4.0
9   10            草津温泉　千　〜ｓｅｎ〜  16,000円〜    4.2
10  11                アイソネット草津  19,800円〜    4.9
11  12            ビジネスホテル アゼリア  12,000円〜    3.9
12  13           草津温泉　ホテルヴィレッジ  19,800円〜    4.4
13  14       ラビスタ草津ヒルズ（共立リゾート）  43,000円〜    4.6
14  15             草津ナウリゾートホテル  33,000円〜    4.1
15  16           草津温泉　ホテルヴィレッジ  19,800円〜    4.4
16  17                草津温泉　VAN  66,480円〜    4.8
17  18             亀の井ホテル 草津湯畑  25,800円〜    4.1
18  19  フォートリート草津〈2023年4月オープン〉  19,800円〜    4.3
19  20               草津温泉　ふた